# 89. 受容野と3D Gaussian Splatting：空間処理の統一的理解

## 学習目標

このノートブックでは、以下を学びます：

1. **CNNの受容野**と**3DGSのGaussian影響範囲**の類似性
2. **局所的な空間処理**という共通の設計原理
3. **階層的表現**の観点からの統一的理解
4. **NeoVerseプロジェクト**との関連

## 目次

1. [3D Gaussian Splattingの復習](#section1)
2. [受容野とGaussian影響範囲の比較](#section2)
3. [局所性という共通原理](#section3)
4. [階層的表現の視点](#section4)
5. [数学的アナロジー](#section5)
6. [まとめ](#summary)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle, Rectangle, Ellipse
from mpl_toolkits.mplot3d import Axes3D
import japanize_matplotlib

plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

<a id="section1"></a>
## 1. 3D Gaussian Splattingの復習

### 3DGSとは

3D Gaussian Splatting (3DGS)は、シーンを多数の3Dガウス関数の集合として表現する手法です。

各Gaussianは以下のパラメータを持ちます：
- **位置** $\boldsymbol{\mu}$：3D空間での中心座標
- **共分散** $\boldsymbol{\Sigma}$：形状と向きを決定
- **不透明度** $\alpha$：透明度
- **色（SH係数）**：球面調和関数で表現

In [ ]:
def visualize_3d_gaussian():
    """3D Gaussianの可視化（2Dスライス）"""
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    # 1. 単一のGaussian
    ax = axes[0]
    x = np.linspace(-3, 3, 100)
    y = np.linspace(-3, 3, 100)
    X, Y = np.meshgrid(x, y)
    
    # 2D Gaussian
    mu = np.array([0, 0])
    sigma = np.array([[1, 0.3], [0.3, 0.5]])  # 楕円形
    
    pos = np.dstack((X, Y))
    sigma_inv = np.linalg.inv(sigma)
    diff = pos - mu
    Z = np.exp(-0.5 * np.sum(diff @ sigma_inv * diff, axis=2))
    
    im = ax.contourf(X, Y, Z, levels=20, cmap='hot')
    ax.set_title('単一の3D Gaussian\n（2Dスライス）', fontsize=14)
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_aspect('equal')
    
    # 影響範囲を示す楕円
    from matplotlib.patches import Ellipse
    eigenvalues, eigenvectors = np.linalg.eig(sigma)
    angle = np.degrees(np.arctan2(eigenvectors[1, 0], eigenvectors[0, 0]))
    for n_sigma in [1, 2, 3]:
        ellipse = Ellipse(mu, 2*n_sigma*np.sqrt(eigenvalues[0]), 
                         2*n_sigma*np.sqrt(eigenvalues[1]), angle,
                         fill=False, color='cyan', linestyle='--', linewidth=2)
        ax.add_patch(ellipse)
    
    # 2. 複数のGaussianの合成
    ax = axes[1]
    np.random.seed(42)
    n_gaussians = 5
    Z_total = np.zeros_like(Z)
    
    centers = np.random.uniform(-2, 2, (n_gaussians, 2))
    colors_list = plt.cm.Set1(np.linspace(0, 1, n_gaussians))
    
    for i, center in enumerate(centers):
        scale = np.random.uniform(0.3, 0.8)
        sigma_i = np.eye(2) * scale
        diff_i = pos - center
        sigma_inv_i = np.linalg.inv(sigma_i)
        Z_i = np.exp(-0.5 * np.sum(diff_i @ sigma_inv_i * diff_i, axis=2))
        Z_total += Z_i
        
        # 中心をマーク
        ax.scatter(center[0], center[1], c=[colors_list[i]], s=100, marker='x', linewidths=2)
    
    ax.contourf(X, Y, Z_total, levels=20, cmap='viridis', alpha=0.8)
    ax.set_title('複数Gaussianの合成\n（3DGSの本質）', fontsize=14)
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_aspect('equal')
    
    # 3. レンダリングへの影響
    ax = axes[2]
    # 1Dでの説明
    x_1d = np.linspace(-3, 3, 200)
    gaussians_1d = []
    centers_1d = [-1.5, 0, 1.2]
    sigmas_1d = [0.5, 0.3, 0.6]
    
    for c, s in zip(centers_1d, sigmas_1d):
        g = np.exp(-0.5 * ((x_1d - c) / s)**2)
        gaussians_1d.append(g)
        ax.plot(x_1d, g, '--', alpha=0.5, label=f'G(μ={c})')
    
    total = sum(gaussians_1d)
    ax.plot(x_1d, total, 'k-', linewidth=2, label='合成結果')
    ax.fill_between(x_1d, 0, total, alpha=0.3)
    
    ax.set_title('Gaussianの加算合成\n（各点への寄与）', fontsize=14)
    ax.set_xlabel('位置')
    ax.set_ylabel('強度')
    ax.legend(fontsize=10)
    ax.grid(True, alpha=0.3)
    
    plt.suptitle('3D Gaussian Splatting: Gaussianの集合でシーンを表現', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

visualize_3d_gaussian()

### 3DGSの核心的アイデア

1. **各Gaussianは有限の影響範囲を持つ**（通常3σ以内）
2. **レンダリング時、各ピクセルには近くのGaussianのみが寄与**
3. **Gaussianの形状（共分散）が「見え方」を決定**

この「局所的な影響範囲」という概念は、CNNの受容野と深く関連しています。

<a id="section2"></a>
## 2. 受容野とGaussian影響範囲の比較

### 概念的な対応関係

In [ ]:
def visualize_rf_vs_gaussian():
    """受容野とGaussian影響範囲の比較"""
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # CNN受容野
    ax = axes[0]
    img_size = 15
    
    # グリッドを描画
    for i in range(img_size + 1):
        ax.axhline(y=i, color='lightgray', linewidth=0.5)
        ax.axvline(x=i, color='lightgray', linewidth=0.5)
    
    # 受容野を円形で近似表示
    center = img_size // 2
    rf_sizes = [3, 5, 7]  # 異なる層の受容野
    colors = ['red', 'orange', 'yellow']
    alphas = [0.3, 0.2, 0.1]
    
    for rf, color, alpha in zip(reversed(rf_sizes), reversed(colors), reversed(alphas)):
        rect = Rectangle((center - rf//2, center - rf//2), rf, rf,
                         linewidth=2, edgecolor=color, facecolor=color, alpha=alpha)
        ax.add_patch(rect)
    
    # 出力ニューロン
    ax.scatter(center + 0.5, center + 0.5, c='black', s=100, zorder=5, marker='s')
    ax.text(center + 1, center - 1, '出力\nニューロン', fontsize=10)
    
    ax.set_xlim(-0.5, img_size + 0.5)
    ax.set_ylim(-0.5, img_size + 0.5)
    ax.set_aspect('equal')
    ax.invert_yaxis()
    ax.set_title('CNN: 受容野\n（矩形の影響範囲）', fontsize=14, fontweight='bold')
    ax.set_xlabel('入力画像の空間次元', fontsize=11)
    
    # 凡例
    from matplotlib.patches import Patch
    legend_elements = [Patch(facecolor=c, alpha=0.5, label=f'層{i+1} (RF={rf})')
                      for i, (rf, c) in enumerate(zip(rf_sizes, colors))]
    ax.legend(handles=legend_elements, loc='upper right', fontsize=10)
    
    # 3DGS Gaussian影響範囲
    ax = axes[1]
    
    # Gaussian分布を可視化
    x = np.linspace(-3, 3, 100)
    y = np.linspace(-3, 3, 100)
    X, Y = np.meshgrid(x, y)
    
    # 楕円形のGaussian
    sigma = np.array([[1.0, 0.3], [0.3, 0.6]])
    sigma_inv = np.linalg.inv(sigma)
    pos = np.dstack((X, Y))
    Z = np.exp(-0.5 * np.sum(pos @ sigma_inv * pos, axis=2))
    
    im = ax.contourf(X, Y, Z, levels=20, cmap='Reds', alpha=0.7)
    
    # σ等高線
    eigenvalues, eigenvectors = np.linalg.eig(sigma)
    angle = np.degrees(np.arctan2(eigenvectors[1, 0], eigenvectors[0, 0]))
    
    for n_sigma, color in zip([1, 2, 3], ['darkred', 'red', 'salmon']):
        ellipse = Ellipse((0, 0), 2*n_sigma*np.sqrt(eigenvalues[0]), 
                         2*n_sigma*np.sqrt(eigenvalues[1]), angle,
                         fill=False, color=color, linestyle='--', linewidth=2,
                         label=f'{n_sigma}σ')
        ax.add_patch(ellipse)
    
    ax.scatter(0, 0, c='black', s=100, zorder=5, marker='o')
    ax.text(0.3, -0.3, 'Gaussian\n中心', fontsize=10)
    
    ax.set_xlim(-3, 3)
    ax.set_ylim(-3, 3)
    ax.set_aspect('equal')
    ax.set_title('3DGS: Gaussian影響範囲\n（楕円形の影響範囲）', fontsize=14, fontweight='bold')
    ax.set_xlabel('空間座標', fontsize=11)
    ax.legend(loc='upper right', fontsize=10)
    
    plt.suptitle('受容野 vs Gaussian影響範囲：局所性の表現', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

visualize_rf_vs_gaussian()

In [ ]:
def print_comparison_table():
    """受容野とGaussianの対応表"""
    print("="*80)
    print("CNN受容野 vs 3DGS Gaussian影響範囲 の対応関係")
    print("="*80)
    
    comparisons = [
        ('概念', '出力ニューロンが見る入力領域', 'Gaussianが影響を及ぼす空間領域'),
        ('形状', '矩形（正方形）', '楕円形（共分散で決定）'),
        ('サイズ決定', 'カーネルサイズ、層の深さ', '共分散行列（σ）'),
        ('重み分布', '学習されたカーネル重み', 'Gaussianの減衰関数'),
        ('中心の重要度', '特に指定なし（一様に見える）', '中心が最大、周辺で減衰'),
        ('階層性', '層を重ねてRFが成長', '異なるスケールのGaussianを使用'),
        ('学習', 'カーネル重みを学習', 'μ, Σ, α, 色を最適化'),
    ]
    
    print(f"\n{'観点':<20} {'CNN受容野':<30} {'3DGS Gaussian':<30}")
    print("-"*80)
    for aspect, cnn, gs in comparisons:
        print(f"{aspect:<20} {cnn:<30} {gs:<30}")

print_comparison_table()

<a id="section3"></a>
## 3. 局所性という共通原理

### なぜ局所的な処理が有効なのか？

CNNと3DGSは、どちらも**局所性の仮定**に基づいています。

1. **空間的相関**: 近くのピクセル/点は似た値を持つことが多い
2. **計算効率**: 全要素間の相互作用を計算するより効率的
3. **汎化性能**: 局所パターンは画像内のどこにでも現れうる

In [ ]:
def visualize_locality_principle():
    """局所性原理の可視化"""
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    
    # 1. 空間的相関の可視化
    ax = axes[0, 0]
    np.random.seed(42)
    # 相関のある2D信号を生成
    from scipy.ndimage import gaussian_filter
    noise = np.random.rand(50, 50)
    smooth = gaussian_filter(noise, sigma=5)
    
    ax.imshow(smooth, cmap='viridis')
    ax.set_title('空間的相関のある信号\n（近くのピクセルは似た値）', fontsize=12)
    ax.axis('off')
    
    # 相関を示す矢印
    ax.annotate('', xy=(30, 25), xytext=(20, 25),
               arrowprops=dict(arrowstyle='<->', color='white', lw=2))
    ax.text(25, 22, '相関', color='white', fontsize=10, ha='center')
    
    # 2. CNNでの局所処理
    ax = axes[0, 1]
    # 5x5のグリッド
    for i in range(6):
        ax.axhline(y=i, color='gray', linewidth=0.5)
        ax.axvline(x=i, color='gray', linewidth=0.5)
    
    # 3x3カーネル
    rect = Rectangle((1, 1), 3, 3, linewidth=3, edgecolor='red', 
                     facecolor='lightyellow', alpha=0.7)
    ax.add_patch(rect)
    
    # カーネル重み
    weights = np.array([[1, 2, 1], [2, 4, 2], [1, 2, 1]]) / 16
    for i in range(3):
        for j in range(3):
            ax.text(1.5 + j, 1.5 + i, f'{weights[i,j]:.2f}', 
                   ha='center', va='center', fontsize=10)
    
    ax.set_xlim(-0.5, 5.5)
    ax.set_ylim(-0.5, 5.5)
    ax.set_aspect('equal')
    ax.invert_yaxis()
    ax.set_title('CNN: 局所カーネル\n（Gaussian blur風の重み）', fontsize=12)
    ax.axis('off')
    
    # 3. 3DGSでの局所処理
    ax = axes[0, 2]
    x = np.linspace(-2, 2, 100)
    y = np.linspace(-2, 2, 100)
    X, Y = np.meshgrid(x, y)
    Z = np.exp(-0.5 * (X**2 + Y**2))
    
    ax.contourf(X, Y, Z, levels=20, cmap='Reds', alpha=0.7)
    ax.scatter(0, 0, c='black', s=100, zorder=5)
    
    for r in [0.5, 1, 1.5]:
        circle = Circle((0, 0), r, fill=False, color='darkred', 
                        linestyle='--', linewidth=1.5)
        ax.add_patch(circle)
    
    ax.set_xlim(-2, 2)
    ax.set_ylim(-2, 2)
    ax.set_aspect('equal')
    ax.set_title('3DGS: Gaussian減衰\n（中心から離れると影響減少）', fontsize=12)
    
    # 4. 計算効率の比較
    ax = axes[1, 0]
    n_values = [10, 50, 100, 500, 1000]
    
    # 全結合: O(n^2)
    fc_ops = [n**2 for n in n_values]
    # 局所: O(n * k^2) where k << n
    k = 3
    local_ops = [n * k**2 for n in n_values]
    
    ax.plot(n_values, fc_ops, 'r-o', label='全結合 O(n²)', linewidth=2)
    ax.plot(n_values, local_ops, 'b-s', label='局所処理 O(n×k²)', linewidth=2)
    ax.set_xlabel('入力サイズ n', fontsize=12)
    ax.set_ylabel('計算量', fontsize=12)
    ax.set_title('計算効率: 局所処理の優位性', fontsize=12)
    ax.legend(fontsize=10)
    ax.set_yscale('log')
    ax.grid(True, alpha=0.3)
    
    # 5. 汎化の可視化
    ax = axes[1, 1]
    # 同じパターンが異なる位置に
    img = np.zeros((20, 20))
    pattern = np.array([[0, 1, 0], [1, 1, 1], [0, 1, 0]])  # 十字
    
    positions = [(2, 2), (10, 5), (5, 14), (14, 12)]
    for y, x in positions:
        img[y:y+3, x:x+3] = pattern
    
    ax.imshow(img, cmap='gray_r')
    ax.set_title('同じ局所パターンが\n異なる位置に出現', fontsize=12)
    ax.axis('off')
    
    # 6. 局所から大域へ
    ax = axes[1, 2]
    ax.text(0.5, 0.9, '局所 → 大域 への階層構築', fontsize=14, 
           ha='center', transform=ax.transAxes, fontweight='bold')
    
    levels = ['ピクセル', 'エッジ', 'テクスチャ', 'パーツ', 'オブジェクト']
    for i, level in enumerate(levels):
        y = 0.7 - i * 0.15
        ax.text(0.3, y, level, fontsize=12, ha='right', transform=ax.transAxes)
        ax.text(0.35, y, '→', fontsize=14, ha='center', transform=ax.transAxes)
        
        # RF/Gaussian sizeの概念図
        size = 0.05 + i * 0.03
        rect = Rectangle((0.4 - size/2, y - size/2), size, size,
                         transform=ax.transAxes, linewidth=1,
                         edgecolor='blue', facecolor='lightblue', alpha=0.7)
        ax.add_patch(rect)
    
    ax.text(0.7, 0.5, 'CNN: 層を重ねてRF拡大\n\n3DGS: 異なるスケールの\nGaussianを組み合わせ', 
           fontsize=11, ha='center', va='center', transform=ax.transAxes,
           bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.axis('off')
    ax.set_title('階層的な表現構築', fontsize=12)
    
    plt.suptitle('局所性：CNNと3DGSの共通原理', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

visualize_locality_principle()

<a id="section4"></a>
## 4. 階層的表現の視点

### CNNの階層的特徴

CNNでは、層を重ねることで：
1. 受容野が徐々に拡大
2. 抽象度が増加（エッジ → パーツ → オブジェクト）

### 3DGSの「階層的」表現

3DGSでは、異なるスケールのGaussianが：
1. 小さいGaussian: 細部・エッジを表現
2. 大きいGaussian: なめらかな領域・背景を表現

In [ ]:
def visualize_multi_scale_gaussians():
    """マルチスケールGaussianの役割"""
    fig, axes = plt.subplots(1, 4, figsize=(18, 4))
    
    x = np.linspace(-5, 5, 200)
    y = np.linspace(-5, 5, 200)
    X, Y = np.meshgrid(x, y)
    
    # 異なるスケールのGaussian
    scales = [0.3, 0.8, 1.5, 3.0]
    titles = ['小スケール\n（細部・エッジ）', '中小スケール\n（テクスチャ）', 
              '中大スケール\n（領域）', '大スケール\n（背景）']
    
    for idx, (scale, title) in enumerate(zip(scales, titles)):
        ax = axes[idx]
        
        # 複数のGaussianを配置
        np.random.seed(42)
        n_gaussians = max(1, int(20 / scale))  # 小さいスケールほど多く
        
        Z_total = np.zeros_like(X)
        for _ in range(n_gaussians):
            cx, cy = np.random.uniform(-3, 3, 2)
            Z = np.exp(-0.5 * ((X - cx)**2 + (Y - cy)**2) / scale**2)
            Z_total += Z
        
        ax.imshow(Z_total, extent=[-5, 5, -5, 5], cmap='hot', origin='lower')
        ax.set_title(f'{title}\nσ ≈ {scale}', fontsize=12)
        ax.axis('off')
    
    plt.suptitle('3DGS: マルチスケールGaussianによる表現', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

visualize_multi_scale_gaussians()

In [ ]:
def visualize_hierarchical_comparison():
    """CNNと3DGSの階層的表現の比較"""
    fig, axes = plt.subplots(2, 4, figsize=(18, 9))
    
    # 上段: CNNの階層
    cnn_labels = ['入力\n(RF=1)', '層1-2\n(RF=5)', '層3-4\n(RF=13)', '層5-6\n(RF=29)']
    rf_sizes = [1, 5, 13, 29]
    
    for idx, (label, rf) in enumerate(zip(cnn_labels, rf_sizes)):
        ax = axes[0, idx]
        
        # 入力画像サイズに対する受容野を可視化
        img_size = 32
        img = np.zeros((img_size, img_size))
        
        center = img_size // 2
        half_rf = min(rf, img_size) // 2
        
        # 受容野領域を1に
        img[center-half_rf:center+half_rf+1, center-half_rf:center+half_rf+1] = 1
        
        ax.imshow(img, cmap='Blues', vmin=0, vmax=1)
        ax.set_title(label, fontsize=12)
        ax.axis('off')
        
        # 受容野サイズを表示
        if rf < img_size:
            ax.text(0.5, -0.1, f'RF={rf}×{rf}', transform=ax.transAxes, 
                   ha='center', fontsize=11)
    
    axes[0, 0].set_ylabel('CNN\n階層的特徴', fontsize=12, rotation=0, ha='right', va='center')
    
    # 下段: 3DGSの異なるスケール
    sigma_values = [0.5, 1.5, 3.0, 6.0]
    gs_labels = ['小σ\n(細部)', '中小σ', '中大σ', '大σ\n(全体)']
    
    x = np.linspace(-5, 5, 100)
    y = np.linspace(-5, 5, 100)
    X, Y = np.meshgrid(x, y)
    
    for idx, (label, sigma) in enumerate(zip(gs_labels, sigma_values)):
        ax = axes[1, idx]
        
        Z = np.exp(-0.5 * (X**2 + Y**2) / sigma**2)
        
        ax.imshow(Z, cmap='Reds', vmin=0, vmax=1, extent=[-5, 5, -5, 5])
        ax.set_title(label, fontsize=12)
        ax.axis('off')
        
        ax.text(0.5, -0.1, f'σ={sigma}', transform=ax.transAxes, 
               ha='center', fontsize=11)
    
    axes[1, 0].set_ylabel('3DGS\nGaussianスケール', fontsize=12, rotation=0, ha='right', va='center')
    
    # 対応関係を示す矢印
    for i in range(4):
        fig.patches.extend([plt.matplotlib.patches.FancyArrowPatch(
            (0.18 + i*0.22, 0.5), (0.18 + i*0.22, 0.48),
            arrowstyle='->', mutation_scale=15, color='green',
            transform=fig.transFigure, connectionstyle='arc3,rad=0')])
    
    plt.suptitle('CNN受容野と3DGS Gaussianスケールの対応', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.subplots_adjust(hspace=0.3)
    plt.show()

visualize_hierarchical_comparison()

<a id="section5"></a>
## 5. 数学的アナロジー

### CNNの畳み込み

$$y(x, y) = \sum_{i,j \in \text{RF}} w_{i,j} \cdot f(x+i, y+j)$$

- $w_{i,j}$: 学習されたカーネル重み
- 矩形領域（受容野）内での重み付き和

### 3DGSのレンダリング

$$C(\mathbf{p}) = \sum_{k} c_k \cdot \alpha_k \cdot \mathcal{G}(\mathbf{p}; \boldsymbol{\mu}_k, \boldsymbol{\Sigma}_k) \cdot T_k$$

- $\mathcal{G}$: Gaussian関数（重み）
- 影響範囲内のGaussianの重み付き合成

In [ ]:
def demonstrate_mathematical_analogy():
    """数学的アナロジーのデモンストレーション"""
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    # 1. CNNの重み付き和
    ax = axes[0]
    
    # 入力パッチ
    np.random.seed(42)
    patch = np.random.rand(3, 3)
    
    # カーネル（Gaussian風）
    kernel = np.array([[1, 2, 1], [2, 4, 2], [1, 2, 1]]) / 16
    
    # 畳み込み出力
    output = np.sum(patch * kernel)
    
    # 可視化
    for i in range(3):
        for j in range(3):
            rect = Rectangle((j, 2-i), 1, 1, linewidth=1, 
                             edgecolor='black', facecolor='lightblue')
            ax.add_patch(rect)
            ax.text(j+0.5, 2-i+0.7, f'{patch[i,j]:.2f}', ha='center', fontsize=9)
            ax.text(j+0.5, 2-i+0.3, f'×{kernel[i,j]:.2f}', ha='center', fontsize=8, color='red')
    
    ax.text(3.5, 1.5, f'= Σ =\n{output:.3f}', fontsize=14, ha='center', va='center')
    
    ax.set_xlim(-0.5, 5)
    ax.set_ylim(-0.5, 3.5)
    ax.set_aspect('equal')
    ax.axis('off')
    ax.set_title('CNN: 離散的重み付き和\n$y = \sum w_{ij} \cdot x_{ij}$', fontsize=14)
    
    # 2. Gaussianの連続重み付き
    ax = axes[1]
    
    x = np.linspace(-2, 2, 100)
    y = np.linspace(-2, 2, 100)
    X, Y = np.meshgrid(x, y)
    
    # Gaussian重み
    sigma = 0.8
    W = np.exp(-0.5 * (X**2 + Y**2) / sigma**2)
    
    ax.imshow(W, extent=[-2, 2, -2, 2], cmap='Reds', origin='lower')
    ax.contour(X, Y, W, levels=[0.1, 0.5, 0.9], colors='darkred', linewidths=1)
    
    ax.set_title('3DGS: Gaussian重み関数\n$\mathcal{G}(x,y) = e^{-\\frac{x^2+y^2}{2\sigma^2}}$', fontsize=14)
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    
    # 3. 共通の抽象化
    ax = axes[2]
    
    ax.text(0.5, 0.85, '共通の数学的構造', fontsize=16, ha='center', 
           transform=ax.transAxes, fontweight='bold')
    
    ax.text(0.5, 0.65, r'$\text{output} = \sum_i w_i \cdot \text{input}_i$', 
           fontsize=18, ha='center', transform=ax.transAxes,
           bbox=dict(boxstyle='round', facecolor='lightyellow'))
    
    ax.text(0.5, 0.45, 'CNN: $w_i$ = 学習されたカーネル重み（離散）', 
           fontsize=12, ha='center', transform=ax.transAxes)
    ax.text(0.5, 0.30, '3DGS: $w_i$ = Gaussian関数値（連続）', 
           fontsize=12, ha='center', transform=ax.transAxes)
    
    ax.text(0.5, 0.1, '両者とも「局所的な重み付き集約」', 
           fontsize=14, ha='center', transform=ax.transAxes,
           color='blue', fontweight='bold')
    
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.axis('off')
    
    plt.suptitle('数学的アナロジー: 重み付き集約', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

demonstrate_mathematical_analogy()

### Gaussian Blurとの関係

実は、Gaussian Blurフィルターは、CNNと3DGSの橋渡しとなる概念です：

- **CNNの視点**: Gaussian分布に基づいたカーネル重みでの畳み込み
- **3DGSの視点**: 入力を小さなGaussianの集合として表現し、それを大きなGaussianでぼかす

In [ ]:
def demonstrate_gaussian_blur_connection():
    """Gaussian Blurを通じたCNNと3DGSの接続"""
    from scipy.ndimage import gaussian_filter
    
    fig, axes = plt.subplots(2, 4, figsize=(18, 9))
    
    # テスト画像
    np.random.seed(42)
    img = np.zeros((64, 64))
    # いくつかの点を配置
    for _ in range(20):
        x, y = np.random.randint(5, 59, 2)
        img[y, x] = 1
    
    axes[0, 0].imshow(img, cmap='gray')
    axes[0, 0].set_title('入力（点の集合）', fontsize=12)
    axes[0, 0].axis('off')
    
    # 異なるσでのGaussian Blur
    sigmas = [1, 2, 4]
    for idx, sigma in enumerate(sigmas):
        blurred = gaussian_filter(img, sigma=sigma)
        axes[0, idx+1].imshow(blurred, cmap='gray')
        axes[0, idx+1].set_title(f'Gaussian Blur σ={sigma}', fontsize=12)
        axes[0, idx+1].axis('off')
    
    # 下段: 解釈
    axes[1, 0].text(0.5, 0.5, 'CNN解釈:\n\n各点を中心に\nGaussianカーネルで\n重み付け平均', 
                   fontsize=12, ha='center', va='center', transform=axes[1, 0].transAxes)
    axes[1, 0].axis('off')
    
    for idx, sigma in enumerate(sigmas):
        ax = axes[1, idx+1]
        ax.text(0.5, 0.6, f'σ={sigma}', fontsize=14, ha='center', 
               transform=ax.transAxes, fontweight='bold')
        ax.text(0.5, 0.4, f'RF ≈ {int(6*sigma)}×{int(6*sigma)}', fontsize=12, 
               ha='center', transform=ax.transAxes, color='blue')
        ax.text(0.5, 0.2, '3DGS解釈:\n各点のGaussianが\nこのσで「にじむ」', 
               fontsize=10, ha='center', transform=ax.transAxes)
        ax.axis('off')
    
    plt.suptitle('Gaussian Blur: CNNと3DGSの架け橋', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

demonstrate_gaussian_blur_connection()

<a id="summary"></a>
## 6. まとめ

### CNNの受容野と3DGS Gaussianの共通点

1. **局所性**: 両者とも有限の空間範囲内で情報を集約
2. **重み付き集約**: 位置に応じた重みで入力を合成
3. **階層/マルチスケール**: 異なるスケールの特徴を捉える
4. **学習可能**: CNNはカーネル、3DGSはGaussianパラメータを最適化

### 違い

| 観点 | CNN受容野 | 3DGS Gaussian |
|------|----------|---------------|
| 形状 | 矩形 | 楕円 |
| 重み分布 | 学習される（任意） | Gaussian（指数減衰） |
| 配置 | 規則的グリッド | 不規則（学習で決定） |
| 次元 | 主に2D | 3D |

### NeoVerseプロジェクトへの示唆

CNNと3DGSの類似性を理解することで：
- 3DGSの「局所性」の意味がより深く理解できる
- CNNで学んだ概念（受容野、階層的特徴）を3DGSに応用できる
- 将来的なNeural 3DGS（学習ベースの3DGS）の理解が容易になる

In [ ]:
def summary_diagram():
    """統合的なまとめ図"""
    fig, ax = plt.subplots(figsize=(16, 10))
    
    # 中央: 共通原理
    ax.add_patch(plt.Circle((0.5, 0.5), 0.15, color='gold', alpha=0.5))
    ax.text(0.5, 0.5, '局所的\n重み付き\n集約', fontsize=14, ha='center', va='center',
           fontweight='bold')
    
    # 左: CNN
    ax.add_patch(plt.Circle((0.2, 0.5), 0.12, color='lightblue', alpha=0.5))
    ax.text(0.2, 0.5, 'CNN\n受容野', fontsize=12, ha='center', va='center')
    
    # 右: 3DGS
    ax.add_patch(plt.Circle((0.8, 0.5), 0.12, color='lightcoral', alpha=0.5))
    ax.text(0.8, 0.5, '3DGS\nGaussian', fontsize=12, ha='center', va='center')
    
    # 接続線
    ax.annotate('', xy=(0.35, 0.5), xytext=(0.32, 0.5),
               arrowprops=dict(arrowstyle='<->', color='blue', lw=2))
    ax.annotate('', xy=(0.68, 0.5), xytext=(0.65, 0.5),
               arrowprops=dict(arrowstyle='<->', color='red', lw=2))
    
    # CNN特性（左側）
    cnn_features = ['矩形領域', '離散的重み', 'グリッド配置', '2D主体']
    for i, feat in enumerate(cnn_features):
        ax.text(0.05, 0.75 - i*0.1, f'• {feat}', fontsize=11)
    
    # 3DGS特性（右側）
    gs_features = ['楕円領域', '連続減衰', '自由配置', '3D空間']
    for i, feat in enumerate(gs_features):
        ax.text(0.85, 0.75 - i*0.1, f'• {feat}', fontsize=11)
    
    # 共通特性（中央下）
    common = ['空間的局所性', '階層的/マルチスケール表現', '学習による最適化', '効率的な計算']
    ax.text(0.5, 0.2, '共通原理', fontsize=14, ha='center', fontweight='bold')
    for i, feat in enumerate(common):
        ax.text(0.5, 0.15 - i*0.04, f'✓ {feat}', fontsize=11, ha='center')
    
    # タイトル
    ax.text(0.5, 0.95, 'CNN受容野と3DGS Gaussian：統一的理解', fontsize=18, 
           ha='center', fontweight='bold')
    
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.axis('off')
    
    plt.tight_layout()
    plt.show()

summary_diagram()

### 次のセクション

Section Bでは受容野について学びました。次のSection Cでは、**帰納バイアス（Inductive Bias）**について深く掘り下げます。

- 重み共有（Weight Sharing）の意味
- 平行移動不変性（Translation Invariance）
- MLPとCNNの比較
- 帰納バイアスとモデルの汎化性能